<a href="https://colab.research.google.com/github/TOM-BOHN/SFDC-User-Permissions-AI/blob/main/Notebooks/SFDC_User_Permission_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Install the Python SDK.

In [1]:
!git clone https://github.com/TOM-BOHN/SFDC-User-Permissions-AI.git

fatal: destination path 'SFDC-User-Permissions-AI' already exists and is not an empty directory.


In [2]:
!pip install -Uq "google-genai==1.7.0"

In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

###################################

import pandas as pd
import enum

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

### Automated retry

This codelab sends a lot of requests, so set up an automatic retry
that ensures your requests are retried when per-minute quota is reached.

In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [6]:
url = "https://raw.githubusercontent.com/TOM-BOHN/SFDC-User-Permissions-AI/refs/heads/main/Inputs/User_Permission_Reference_Data__Sample.csv"
df = pd.read_csv(url)
df.head()

,Permission Name,API Name,Description
0,Access Data Cloud Data Explorer,AccessCdpDataExplorer,Allows user access Data Cloud Data Explorer.
1,Administer territory operations,ManageTerritories,Prerequisite user permission for a user to man...
2,Allow sending of List Emails,ListEmailSend,"Allow users to create, edit and send List Emails"
3,Api Only User,ApiUserOnly,Access Salesforce.com only through a Salesforc...
4,Author Apex,AuthorApex,Create Apex classes and triggers.


In [9]:
with open('/content/SFDC-User-Permissions-AI/Prompts/prompt_user_perm_risk_rating.md', 'r') as f:
    PROMPT_USER_PERM_RISK_RATING = f.read()

print(PROMPT_USER_PERM_RISK_RATING)

# Permission Risk Evaluation Prompt Template  
# --------------------------------------------------
# This template can be imported and formatted with the specific
# `permission_name` and `permission_description` variables to create
# a concrete evaluation prompt for any Salesforce permission.
# --------------------------------------------------

PERMISSION_RISK_PROMPT = """
# Instruction
You are a **Salesforce security risk assessor**.
Your task is to evaluate the **inherent risk level** of a Salesforce permission (or capability) when granted to a user.
We will provide you with the permission name and a short description of what it allows.
Analyze the permission against the **Evaluation Criteria** below and assign one of the five **Risk Levels** defined in the Rating Rubric.
Give step‑by‑step reasoning for your decision, citing the specific criteria that most influenced your rating.

# Evaluation

## Metric Definition
**Permission Risk** [aka weighted_score] measures the potential neg

In [10]:
# Define a structured enum class to capture the result.
class RiskRating(enum.Enum):
  MISSION_CRITICAL = '5'
  RESTRICTED = '4'
  SENSITIVE = '3'
  CONTROLLED = '2'
  GENERAL = '1'

def eval_summary(PROMPT, name, api_name, description):
  """Evaluate the generated summary against the prompt used."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=PROMPT.format(
          permission_name = name
        , permission_description = description
      )
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=RiskRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(
    PROMPT=PROMPT_USER_PERM_RISK_RATING
  , name=df['Permission Name'][0]
  , api_name=df['API Name'][0]
  , description=df['Description'][0]
)

Markdown(text_eval)

```json
{
  "risk_tier": "Sensitive",
  "risk_rating": "3",
  "weighted_score": 3.0,
  "scores": {
    "Data_Sensitivity": 3,
    "Scope_of_Impact": 3,
    "Configurational_Authority": 2,
    "External_Data_Exposure": 2,
    "Regulatory_Obligation": 3,
    "Segregation_of_Duties": 3,
    "Auditability": 4,
    "Reversibility": 4
  },
  "rationale": "Access to Data Cloud Data Explorer allows users to potentially view and analyze sensitive data within Data Cloud. The scope of impact is limited to the data accessible through the Data Explorer, but still presents a moderate risk. Regulatory obligations may arise depending on the type of data stored and accessed within Data Cloud.",
  "confidence": "High"
}
```

In [11]:
struct_eval

<RiskRating.SENSITIVE: '3'>

In [ ]:
total_records = len(df)

for i in range(2):
  print(f'Analyzing Permission {i+1} of {total_records}...')
  print('Name:       ', df['Permission Name'][i])
  print('API Name:   ', df['API Name'][i])
  print('Description:', df['Description'][i])
  print('--------------------')

  # Substitute variables into the prompt
  prompt_with_vars = prompt_user_perm_risk_rating.format(
    permission_name=df['Permission Name'][i],
    permission_description=df['Description'][i]
  )